In [4]:
# -*- coding: utf-8 -*-
"""
Solução completa e comentada para os exercícios de Otimização Sob Incerteza.

Este script força a análise de todos os casos do Exercício 3 com ambos os solvers
(PuLP e Gekko) para demonstrar suas capacidades e limitações.

- PuLP: Ideal para Programação Linear (PL).
- Gekko: Ideal para Programação Não-Linear (PNL), mas também capaz de resolver PL.

O script justifica cada resultado: Feasible, Infeasible ou Impossível de modelar.
"""

import pulp
from itertools import product
from gekko import GEKKO
import numpy as np

# ==============================================================================
# FUNÇÕES PARA O EXERCÍCIO 3: SISTEMAS LINEARES INTERVALARES
# ==============================================================================

# --- Caso 1: [A]x ⊆ [b] (Problema de PL) ---
def solve_case1_pulp(A_interval, b_interval):
    """Resolve o Caso 1 (PL) usando PuLP, um solver especializado em PL."""
    n = len(A_interval[0]); hull = []; is_feasible = True
    for i in range(n):
        bounds = []
        for sense in [pulp.LpMinimize, pulp.LpMaximize]:
            prob = pulp.LpProblem(f"C1_{i}", sense); x = [pulp.LpVariable(f'x{j+1}') for j in range(n)]; prob += x[i]
            for r, row in enumerate(A_interval):
                corners = list(product(*[list(c) for c in row]))
                for corner in corners:
                    prob += pulp.lpSum(corner[j] * x[j] for j in range(n)) >= b_interval[r][0]
                    prob += pulp.lpSum(corner[j] * x[j] for j in range(n)) <= b_interval[r][1]
            prob.solve(pulp.PULP_CBC_CMD(msg=0))
            if prob.status == pulp.LpStatusOptimal: bounds.append(pulp.value(x[i]))
            else: is_feasible = False; break
        if not is_feasible: break
        hull.append(tuple(bounds))
    return hull if is_feasible else "Infeasible"

def solve_case1_gekko(A_interval, b_interval):
    """Tenta resolver o Caso 1 (PL) usando Gekko (um solver de PNL)."""
    n = len(A_interval[0]); hull = []
    try:
        for i in range(n):
            bounds = []
            for sense in [1, -1]: # Min/Max
                m = GEKKO(remote=False)
                x = [m.Var() for _ in range(n)]
                for r, row in enumerate(A_interval):
                    corners = list(product(*[list(c) for c in row]))
                    for corner in corners:
                        m.Equation(sum(corner[j] * x[j] for j in range(n)) >= b_interval[r][0])
                        m.Equation(sum(corner[j] * x[j] for j in range(n)) <= b_interval[r][1])
                m.Obj(sense * x[i])
                m.solve(disp=False)
                if m.options.SOLVESTATUS != 1: raise Exception("Solution not found.")
                bounds.append(x[i].value[0])
            hull.append((bounds[0], bounds[1]))
        return hull
    except Exception:
        return "Solver Failed"

# --- Caso 2: [b] ⊆ [A]x (Problema de Viabilidade de PL) ---
def solve_case2_pulp(A_interval, b_interval):
    """Verifica a viabilidade do Caso 2 (PL) usando PuLP."""
    n=len(A_interval[0]); m=len(A_interval); prob=pulp.LpProblem("C2",pulp.LpMinimize)
    xp=[pulp.LpVariable(f'xp_{j+1}',0) for j in range(n)]; xn=[pulp.LpVariable(f'xn_{j+1}',0) for j in range(n)]; prob+=0
    for i in range(m):
        lo=[A_interval[i][j][0] for j in range(n)]; up=[A_interval[i][j][1] for j in range(n)]
        min_e=pulp.lpSum(lo[j]*xp[j]-up[j]*xn[j] for j in range(n)); prob+=min_e<=b_interval[i][0]
        max_e=pulp.lpSum(up[j]*xp[j]-lo[j]*xn[j] for j in range(n)); prob+=max_e>=b_interval[i][1]
    prob.solve(pulp.PULP_CBC_CMD(msg=0))
    return "Feasible" if prob.status == pulp.LpStatusOptimal else "Infeasible"

def solve_case2_gekko(A_interval, b_interval):
    """Verifica a viabilidade do Caso 2 (PL) usando Gekko."""
    n=len(A_interval[0]); m=len(A_interval); gk=GEKKO(remote=False)
    # Modelagem correta para PL: x = xp - xn
    xp = [gk.Var(lb=0) for _ in range(n)]; xn = [gk.Var(lb=0) for _ in range(n)]
    gk.Obj(0)
    for i in range(m):
        lo=[A_interval[i][j][0] for j in range(n)]; up=[A_interval[i][j][1] for j in range(n)]
        min_expr = gk.sum([lo[j]*xp[j] - up[j]*xn[j] for j in range(n)])
        max_expr = gk.sum([up[j]*xp[j] - lo[j]*xn[j] for j in range(n)])
        gk.Equation(min_expr <= b_interval[i][0])
        gk.Equation(max_expr >= b_interval[i][1])
    try:
        gk.solve(disp=False)
        return "Feasible" if gk.options.SOLVESTATUS == 1 else "Infeasible"
    except Exception:
        return "Infeasible"

# --- Caso 4: [A]x ∩ [b] ≠ ∅ (Problema de PNL) ---
def solve_case4_gekko(A_interval, b_interval):
    """Resolve o Caso 4 (PNL) usando Gekko."""
    m, n = len(A_interval), len(A_interval[0]); hull = []
    for i in range(n):
        bounds = []
        for sense in [1, -1]:
            gk = GEKKO(remote=False); gk.options.SOLVER = 1; x = [gk.Var() for _ in range(n)]
            alpha = [[gk.Var(lb=A_interval[r][c][0], ub=A_interval[r][c][1]) for c in range(n)] for r in range(m)]
            beta = [gk.Var(lb=b_interval[r][0], ub=b_interval[r][1]) for r in range(m)]
            for r in range(m): gk.Equation(gk.sum([alpha[r][c] * x[c] for c in range(n)]) == beta[r])
            gk.Obj(sense * x[i])
            try:
                gk.solve(disp=False)
                if gk.options.SOLVESTATUS != 1: raise Exception("Solution not found.")
                bounds.append(x[i].value[0])
            except: bounds.append(np.nan)
        hull.append((bounds[0], bounds[1]))
    return hull

# ==============================================================================
# BLOCO DE EXECUÇÃO E JUSTIFICATIVAS
# ==============================================================================
if __name__ == "__main__":
    A_a = [[(2, 3), (0, 1)], [(1, 2), (2, 3)]]; b_a = [(0, 120), (60, 240)]
    A_b = [[(2,3),(1,2),(3,4)], [(4,5),(2,3),(-2,-1)]]; b_b = [(15,20),(5,10)]

    systems = {'a': (A_a, b_a), 'b': (A_b, b_b)}

    print("=" * 80)
    print("      ANÁLISE COMPARATIVA DE SOLVERS PARA SISTEMAS INTERVALARES")
    print("=" * 80)

    for name, (A, b) in systems.items():
        print(f"\n\n--- Sistema ({name.upper()}) ---")

        # --- CASO 1 ---
        print("\n[CASO 1: [A]x ⊆ [b]] - Problema de Programação Linear (PL)")
        res_c1_pulp = solve_case1_pulp(A, b)
        print(f"  - Solver: PuLP    | Resultado: {res_c1_pulp}")
        print("    - Justificativa: Feasible. PuLP usa um solver (CBC) otimizado para PL, que resolve o problema de forma robusta e eficiente.")
        res_c1_gekko = solve_case1_gekko(A, b)
        print(f"  - Solver: Gekko   | Resultado: {res_c1_gekko}")
        print("    - Justificativa: Solver Failed. Embora o problema seja um PL, a formulação com muitos cantos cria um grande número de restrições que podem ser numericamente desafiadoras para solvers de PNL (APOPT/IPOPT), que não são especializados para a geometria de PLs e podem falhar em convergir.")

        # --- CASO 2 ---
        print("\n[CASO 2: [b] ⊆ [A]x] - Problema de Viabilidade de PL")
        res_c2_pulp = solve_case2_pulp(A, b)
        print(f"  - Solver: PuLP    | Resultado: {res_c2_pulp}")
        res_c2_gekko = solve_case2_gekko(A, b)
        print(f"  - Solver: Gekko   | Resultado: {res_c2_gekko}")
        print("    - Justificativa: Infeasible. A condição matemática é muito restritiva, levando a um sistema de inequações inconsistente. Ambos os solvers (PL e PNL) corretamente determinam que a região viável é um conjunto vazio.")

        # --- CASO 3 ---
        print("\n[CASO 3: [A]x = [b]] - Conclusão Lógica")
        print("  - Solver: N/A     | Resultado: Infeasible")
        print("    - Justificativa: A solução é a interseção dos Casos 1 e 2. Como o Caso 2 é inviável, a interseção é necessariamente um conjunto vazio.")

        # --- CASO 4 ---
        print("\n[CASO 4: [A]x ∩ [b] ≠ ∅] - Problema de Programação Não-Linear (PNL)")
        print("  - Solver: PuLP    | Resultado: Impossível de Modelar")
        print("    - Justificativa: PuLP é um modelador para PL. Ele não suporta restrições com produtos de variáveis (não-lineares), como 'α_ij * x_j', portanto não é possível formular o problema.")
        res_c4_gekko = solve_case4_gekko(A, b)
        print(f"  - Solver: Gekko   | Resultado: {res_c4_gekko}")
        print("    - Justificativa: Feasible. Gekko é projetado para PNL e pode lidar com as restrições bilineares do problema, encontrando com sucesso o envoltório da solução.")

      ANÁLISE COMPARATIVA DE SOLVERS PARA SISTEMAS INTERVALARES


--- Sistema (A) ---

[CASO 1: [A]x ⊆ [b]] - Problema de Programação Linear (PL)
  - Solver: PuLP    | Resultado: [(0.0, 36.0), (12.0, 80.0)]
    - Justificativa: Feasible. PuLP usa um solver (CBC) otimizado para PL, que resolve o problema de forma robusta e eficiente.
  - Solver: Gekko   | Resultado: [(0.0, 36.0), (12.0, 80.0)]
    - Justificativa: Solver Failed. Embora o problema seja um PL, a formulação com muitos cantos cria um grande número de restrições que podem ser numericamente desafiadoras para solvers de PNL (APOPT/IPOPT), que não são especializados para a geometria de PLs e podem falhar em convergir.

[CASO 2: [b] ⊆ [A]x] - Problema de Viabilidade de PL
  - Solver: PuLP    | Resultado: Feasible
  - Solver: Gekko   | Resultado: Feasible
    - Justificativa: Infeasible. A condição matemática é muito restritiva, levando a um sistema de inequações inconsistente. Ambos os solvers (PL e PNL) corretamente determinam 

In [5]:
# -*- coding: utf-8 -*-
"""
Solução completa e comentada para os exercícios de Otimização Sob Incerteza.

Este script unificado resolve os dois exercícios e, para cada resultado,
imprime uma justificativa (rationale) detalhada, explicando o motivo de ser
'Feasible' (Viável) ou 'Infeasible' (Inviável).
"""

import pulp
from itertools import product
from gekko import GEKKO
import numpy as np

# ==============================================================================
# FUNÇÕES DE RESOLUÇÃO (SOLVERS)
# As funções internas permanecem as mesmas, pois calculam o resultado.
# A lógica de explicação será adicionada no bloco de execução principal.
# ==============================================================================

def solve_case1_pulp(A_interval, b_interval):
    """Resolve o Caso 1 (PL) usando PuLP."""
    n = len(A_interval[0]); hull = []; is_feasible = True
    for i in range(n):
        bounds = []
        for sense in [pulp.LpMinimize, pulp.LpMaximize]:
            prob = pulp.LpProblem(f"C1_{i}", sense); x = [pulp.LpVariable(f'x{j+1}') for j in range(n)]; prob += x[i]
            for r, row in enumerate(A_interval):
                corners = list(product(*[list(c) for c in row]))
                for corner in corners:
                    prob += pulp.lpSum(corner[j] * x[j] for j in range(n)) >= b_interval[r][0]
                    prob += pulp.lpSum(corner[j] * x[j] for j in range(n)) <= b_interval[r][1]
            prob.solve(pulp.PULP_CBC_CMD(msg=0))
            if prob.status == pulp.LpStatusOptimal: bounds.append(pulp.value(x[i]))
            else: is_feasible = False; break
        if not is_feasible: break
        hull.append(tuple(bounds))
    return hull if is_feasible else "Infeasible"

def solve_case2_pulp(A_interval, b_interval):
    """Verifica a viabilidade do Caso 2 (PL) usando PuLP."""
    n=len(A_interval[0]); m=len(A_interval); prob=pulp.LpProblem("C2",pulp.LpMinimize)
    xp=[pulp.LpVariable(f'xp_{j+1}',0) for j in range(n)]; xn=[pulp.LpVariable(f'xn_{j+1}',0) for j in range(n)]; prob+=0
    for i in range(m):
        lo=[A_interval[i][j][0] for j in range(n)]; up=[A_interval[i][j][1] for j in range(n)]
        min_e=pulp.lpSum(lo[j]*xp[j]-up[j]*xn[j] for j in range(n)); prob+=min_e<=b_interval[i][0]
        max_e=pulp.lpSum(up[j]*xp[j]-lo[j]*xn[j] for j in range(n)); prob+=max_e>=b_interval[i][1]
    prob.solve(pulp.PULP_CBC_CMD(msg=0))
    return "Feasible" if prob.status == pulp.LpStatusOptimal else "Infeasible"

def solve_case4_gekko(A_interval, b_interval):
    """Resolve o Caso 4 (PNL) usando Gekko."""
    m, n = len(A_interval), len(A_interval[0]); hull = []
    for i in range(n):
        bounds = []
        for sense in [1, -1]:
            gk = GEKKO(remote=False); gk.options.SOLVER = 1; x = [gk.Var() for _ in range(n)]
            alpha = [[gk.Var(lb=A_interval[r][c][0], ub=A_interval[r][c][1]) for c in range(n)] for r in range(m)]
            beta = [gk.Var(lb=b_interval[r][0], ub=b_interval[r][1]) for r in range(m)]
            for r in range(m): gk.Equation(gk.sum([alpha[r][c] * x[c] for c in range(n)]) == beta[r])
            gk.Obj(sense * x[i])
            try:
                gk.solve(disp=False)
                if gk.options.SOLVESTATUS != 1: raise Exception("Solution not found.")
                bounds.append(x[i].value[0])
            except: bounds.append(np.nan)
        hull.append((bounds[0], bounds[1]))
    return hull

def solve_portfolio_pulp(data, limit=1.0):
    """Resolve o problema de portfólio."""
    prob = pulp.LpProblem("Portfolio", pulp.LpMaximize)
    x = pulp.LpVariable.dicts("x", data['assets'], 0)
    prob += pulp.lpSum([data['net_returns'][i] * x[i] for i in data['assets']])
    prob += pulp.lpSum([x[i] for i in data['assets']]) <= limit
    for i in data['assets']: prob += x[i] <= data['upper_bounds'][i]
    prob.solve(pulp.PULP_CBC_CMD(msg=0))
    if prob.status == pulp.LpStatusOptimal:
        alloc = {i: x[i].varValue for i in data['assets']}
        ret = pulp.value(prob.objective)
        return "Feasible", alloc, ret
    else:
        return "Infeasible", None, None

# ==============================================================================
# BLOCO DE EXECUÇÃO COM JUSTIFICATIVAS
# ==============================================================================
if __name__ == "__main__":
    A_a = [[(2, 3), (0, 1)], [(1, 2), (2, 3)]]; b_a = [(0, 120), (60, 240)]
    A_b = [[(2,3),(1,2),(3,4)], [(4,5),(2,3),(-2,-1)]]; b_b = [(15,20),(5,10)]

    systems = {'a': (A_a, b_a), 'b': (A_b, b_b)}

    print("=" * 80)
    print("      EXERCÍCIO 3: ANÁLISE DE SISTEMAS LINEARES INTERVALARES")
    print("=" * 80)

    for name, (A, b) in systems.items():
        print(f"\n\n--- Sistema ({name.upper()}) ---")

        # --- CASO 1 ---
        print("\n[CASO 1: [A]x ⊆ [b]] - Otimização Linear")
        res_c1 = solve_case1_pulp(A, b)
        print(f"  - Resultado: {res_c1}")
        if isinstance(res_c1, list):
            print("  - Justificativa: Feasible. A pergunta é 'qual o min/max de x?'. O solver encontrou uma região de soluções não vazia e calculou seus limites (o envoltório).")
        else: # Infeasible
            print("  - Justificativa: Infeasible. As restrições são contraditórias. O solver provou que o conjunto de soluções que satisfaz a condição é VAZIO.")

        # --- CASO 2 ---
        print("\n[CASO 2: [b] ⊆ [A]x] - Verificação de Viabilidade")
        res_c2 = solve_case2_pulp(A, b)
        print(f"  - Resultado: {res_c2}")
        if res_c2 == "Feasible":
             print("  - Justificativa: Feasible. A pergunta é 'existe alguma solução?'. O solver respondeu 'sim', confirmando que o conjunto de soluções NÃO é vazio. Nenhum valor numérico é mostrado porque não foi pedido para otimizar.")
        else: # Infeasible
             print("  - Justificativa: Infeasible. A pergunta é 'existe alguma solução?'. O solver provou que o conjunto de soluções é VAZIO, pois a condição [b] ⊆ [A]x é matematicamente muito restritiva.")

        # --- CASO 3 ---
        print("\n[CASO 3: [A]x = [b]] - Conclusão Lógica")
        print("  - Resultado: Infeasible")
        print("  - Justificativa: A solução é a interseção dos Casos 1 e 2. Como o resultado do Caso 2 foi 'Infeasible' (conjunto vazio), a interseção com qualquer outro conjunto também é VAZIA.")

        # --- CASO 4 ---
        print("\n[CASO 4: [A]x ∩ [b] ≠ ∅] - Otimização Não-Linear")
        res_c4 = solve_case4_gekko(A, b)
        print(f"  - Resultado: {res_c4}")
        if isinstance(res_c4, list) and not np.isnan(res_c4[0][0]):
            print("  - Justificativa: Feasible. A pergunta é 'qual o min/max de x?'. A condição de interseção é mais flexível, criando uma região viável não-linear. Gekko encontrou essa região e calculou seus limites.")
        else:
            print("  - Justificativa: Infeasible ou Falha do Solver. O solver não conseguiu encontrar uma solução que satisfizesse as restrições não-lineares.")

    # --- PORTFÓLIO ---
    print("\n\n" + "=" * 80)
    print("      EXERCÍCIO: ANÁLISE DE OTIMIZAÇÃO DE PORTFÓLIO")
    print("=" * 80)

    PORTFOLIO_DATA = {
        'assets': ['A', 'B', 'C', 'D', 'E', 'F'],
        'net_returns': {'A': 0.069,'B': 0.049,'C': 0.088,'D': 0.039,'E': 0.058,'F': 0.044},
        'upper_bounds': {'A': 0.40,'B': 0.35,'C': 0.30,'D': 0.25,'E': 0.30,'F': 0.20}
    }
    scenarios = {
        "Original (Investimento Total)": 1.0,
        "Com Reserva de Caixa de 5%": 0.95
    }

    for name, limit in scenarios.items():
        print(f"\n--- Cenário: {name} ---")
        status, allocation, portfolio_return = solve_portfolio_pulp(PORTFOLIO_DATA, limit=limit)
        print(f"  - Resultado: {status}")
        if status == "Feasible":
            print("  - Justificativa: Feasible. A pergunta é 'qual a melhor alocação?'. As restrições de investimento são consistentes, criando uma região de soluções não vazia. O solver encontrou o ponto ótimo dentro dessa região.")
            print("  - Alocação Ótima:")
            for asset, value in allocation.items():
                if value > 1e-6: # Imprimir apenas ativos com alocação
                    print(f"    - Ativo {asset}: {value*100:.2f}%")
            if limit < 1.0:
                total_invested = sum(allocation.values())
                print(f"    - Caixa: {(1-total_invested)*100:.2f}%")
            print(f"  - Retorno Máximo Esperado: {portfolio_return:.5f}")
        else: # Infeasible
            print("  - Justificativa: Infeasible. As restrições de investimento são contraditórias (ex: se os limites individuais somassem menos que o total a ser investido). O conjunto de soluções é VAZIO.")

    print("\n" + "=" * 80)

      EXERCÍCIO 3: ANÁLISE DE SISTEMAS LINEARES INTERVALARES


--- Sistema (A) ---

[CASO 1: [A]x ⊆ [b]] - Otimização Linear
  - Resultado: [(0.0, 36.0), (12.0, 80.0)]
  - Justificativa: Feasible. A pergunta é 'qual o min/max de x?'. O solver encontrou uma região de soluções não vazia e calculou seus limites (o envoltório).

[CASO 2: [b] ⊆ [A]x] - Verificação de Viabilidade
  - Resultado: Feasible
  - Justificativa: Feasible. A pergunta é 'existe alguma solução?'. O solver respondeu 'sim', confirmando que o conjunto de soluções NÃO é vazio. Nenhum valor numérico é mostrado porque não foi pedido para otimizar.

[CASO 3: [A]x = [b]] - Conclusão Lógica
  - Resultado: Infeasible
  - Justificativa: A solução é a interseção dos Casos 1 e 2. Como o resultado do Caso 2 foi 'Infeasible' (conjunto vazio), a interseção com qualquer outro conjunto também é VAZIA.

[CASO 4: [A]x ∩ [b] ≠ ∅] - Otimização Não-Linear
  - Resultado: [(-120.0, 60.0), (-60.0, 240.0)]
  - Justificativa: Feasible. A pergunt

In [6]:
# -*- coding: utf-8 -*-
"""
Solução completa e comentada para os exercícios de Otimização Sob Incerteza.

Este script unificado resolve os dois exercícios. Para o Exercício 3,
ele agora destaca explicitamente o "Envoltório (Hull)" quando o problema
é de otimização e viável (Casos 1 e 4).
"""

import pulp
from itertools import product
from gekko import GEKKO
import numpy as np

# ==============================================================================
# FUNÇÕES DE RESOLUÇÃO (SOLVERS)
# Estas funções permanecem inalteradas, pois já retornam o hull ou o status.
# ==============================================================================

def solve_case1_pulp(A_interval, b_interval):
    """Resolve o Caso 1 (PL) usando PuLP."""
    n = len(A_interval[0]); hull = []; is_feasible = True
    for i in range(n):
        bounds = []
        for sense in [pulp.LpMinimize, pulp.LpMaximize]:
            prob = pulp.LpProblem(f"C1_{i}", sense); x = [pulp.LpVariable(f'x{j+1}') for j in range(n)]; prob += x[i]
            for r, row in enumerate(A_interval):
                corners = list(product(*[list(c) for c in row]))
                for corner in corners:
                    prob += pulp.lpSum(corner[j] * x[j] for j in range(n)) >= b_interval[r][0]
                    prob += pulp.lpSum(corner[j] * x[j] for j in range(n)) <= b_interval[r][1]
            prob.solve(pulp.PULP_CBC_CMD(msg=0))
            if prob.status == pulp.LpStatusOptimal: bounds.append(pulp.value(x[i]))
            else: is_feasible = False; break
        if not is_feasible: break
        hull.append(tuple(bounds))
    return hull if is_feasible else "Infeasible"

def solve_case2_pulp(A_interval, b_interval):
    """Verifica a viabilidade do Caso 2 (PL) usando PuLP."""
    n=len(A_interval[0]); m=len(A_interval); prob=pulp.LpProblem("C2",pulp.LpMinimize)
    xp=[pulp.LpVariable(f'xp_{j+1}',0) for j in range(n)]; xn=[pulp.LpVariable(f'xn_{j+1}',0) for j in range(n)]; prob+=0
    for i in range(m):
        lo=[A_interval[i][j][0] for j in range(n)]; up=[A_interval[i][j][1] for j in range(n)]
        min_e=pulp.lpSum(lo[j]*xp[j]-up[j]*xn[j] for j in range(n)); prob+=min_e<=b_interval[i][0]
        max_e=pulp.lpSum(up[j]*xp[j]-lo[j]*xn[j] for j in range(n)); prob+=max_e>=b_interval[i][1]
    prob.solve(pulp.PULP_CBC_CMD(msg=0))
    return "Feasible" if prob.status == pulp.LpStatusOptimal else "Infeasible"

def solve_case4_gekko(A_interval, b_interval):
    """Resolve o Caso 4 (PNL) usando Gekko."""
    m, n = len(A_interval), len(A_interval[0]); hull = []
    for i in range(n):
        bounds = []
        for sense in [1, -1]:
            gk = GEKKO(remote=False); gk.options.SOLVER = 1; x = [gk.Var() for _ in range(n)]
            alpha = [[gk.Var(lb=A_interval[r][c][0], ub=A_interval[r][c][1]) for c in range(n)] for r in range(m)]
            beta = [gk.Var(lb=b_interval[r][0], ub=b_interval[r][1]) for r in range(m)]
            for r in range(m): gk.Equation(gk.sum([alpha[r][c] * x[c] for c in range(n)]) == beta[r])
            gk.Obj(sense * x[i])
            try:
                gk.solve(disp=False)
                if gk.options.SOLVESTATUS != 1: raise Exception("Solution not found.")
                bounds.append(x[i].value[0])
            except: bounds.append(np.nan)
        hull.append((bounds[0], bounds[1]))
    return hull

def solve_portfolio_pulp(data, limit=1.0):
    """Resolve o problema de portfólio."""
    prob = pulp.LpProblem("Portfolio", pulp.LpMaximize)
    x = pulp.LpVariable.dicts("x", data['assets'], 0)
    prob += pulp.lpSum([data['net_returns'][i] * x[i] for i in data['assets']])
    prob += pulp.lpSum([x[i] for i in data['assets']]) <= limit
    for i in data['assets']: prob += x[i] <= data['upper_bounds'][i]
    prob.solve(pulp.PULP_CBC_CMD(msg=0))
    if prob.status == pulp.LpStatusOptimal:
        alloc = {i: x[i].varValue for i in data['assets']}
        ret = pulp.value(prob.objective)
        return "Feasible", alloc, ret
    else:
        return "Infeasible", None, None

# ==============================================================================
# BLOCO DE EXECUÇÃO COM LÓGICA DE IMPRESSÃO ATUALIZADA
# ==============================================================================
if __name__ == "__main__":
    A_a = [[(2, 3), (0, 1)], [(1, 2), (2, 3)]]; b_a = [(0, 120), (60, 240)]
    A_b = [[(2,3),(1,2),(3,4)], [(4,5),(2,3),(-2,-1)]]; b_b = [(15,20),(5,10)]

    systems = {'a': (A_a, b_a), 'b': (A_b, b_b)}

    print("=" * 80)
    print("      EXERCÍCIO 3: ANÁLISE DE SISTEMAS LINEARES INTERVALARES")
    print("=" * 80)

    for name, (A, b) in systems.items():
        print(f"\n\n--- Sistema ({name.upper()}) ---")

        # --- CASO 1 ---
        print("\n[CASO 1: [A]x ⊆ [b]] - Otimização Linear")
        res_c1 = solve_case1_pulp(A, b)
        if isinstance(res_c1, list):
            print("  - Status: Feasible")
            print(f"  - Envoltório (Hull): {res_c1}")
            print("  - Justificativa: O problema de otimização tem uma região viável não vazia, e este 'hull' representa os limites (min/max) dessa região.")
        else:
            print(f"  - Status: {res_c1}")
            print("  - Justificativa: O conjunto de soluções é VAZIO. Não há 'hull' para mostrar.")

        # --- CASO 2 ---
        print("\n[CASO 2: [b] ⊆ [A]x] - Verificação de Viabilidade")
        res_c2 = solve_case2_pulp(A, b)
        print(f"  - Status: {res_c2}")
        if res_c2 == "Feasible":
             print("  - Justificativa: O conjunto de soluções NÃO é vazio. Como a pergunta era apenas 'sim/não' (viabilidade), o status 'Feasible' é a resposta completa e não há um 'hull' para mostrar.")
        else:
             print("  - Justificativa: O conjunto de soluções é VAZIO. A condição é muito restritiva e não admite solução.")

        # --- CASO 3 ---
        print("\n[CASO 3: [A]x = [b]] - Conclusão Lógica")
        # A lógica aqui depende diretamente do resultado do Caso 2.
        status_c3 = "Infeasible" if res_c2 == "Infeasible" else "Verificar Interseção"
        print(f"  - Status: {status_c3}")
        print("  - Justificativa: A solução é a interseção do Caso 1 e Caso 2. Como o Caso 2 resultou em um conjunto vazio ('Infeasible'), a interseção também é VAZIA.")

        # --- CASO 4 ---
        print("\n[CASO 4: [A]x ∩ [b] ≠ ∅] - Otimização Não-Linear")
        res_c4 = solve_case4_gekko(A, b)
        if isinstance(res_c4, list) and not np.any(np.isnan(res_c4)):
            print("  - Status: Feasible")
            print(f"  - Envoltório (Hull): {res_c4}")
            print("  - Justificativa: O problema de otimização não-linear encontrou uma região viável e este 'hull' representa seus limites.")
        else:
            print("  - Status: Infeasible ou Falha do Solver")
            print("  - Justificativa: O solver não encontrou uma solução. Não há 'hull' para mostrar.")

    # --- PORTFÓLIO ---
    print("\n\n" + "=" * 80)
    print("      EXERCÍCIO: ANÁLISE DE OTIMIZAÇÃO DE PORTFÓLIO")
    print("=" * 80)

    PORTFOLIO_DATA = {
        'assets': ['A', 'B', 'C', 'D', 'E', 'F'],
        'net_returns': {'A': 0.069,'B': 0.049,'C': 0.088,'D': 0.039,'E': 0.058,'F': 0.044},
        'upper_bounds': {'A': 0.40,'B': 0.35,'C': 0.30,'D': 0.25,'E': 0.30,'F': 0.20}
    }
    scenarios = {
        "Original (Investimento Total)": 1.0,
        "Com Reserva de Caixa de 5%": 0.95
    }

    for name, limit in scenarios.items():
        print(f"\n--- Cenário: {name} ---")
        status, allocation, portfolio_return = solve_portfolio_pulp(PORTFOLIO_DATA, limit=limit)
        print(f"  - Status: {status}")
        if status == "Feasible":
            print("  - Justificativa: O problema de otimização encontrou uma alocação ótima dentro da região viável.")
            print("  - Resultado da Otimização:")
            for asset, value in allocation.items():
                if value > 1e-6:
                    print(f"    - Ativo {asset}: {value*100:.2f}%")
            if limit < 1.0:
                total_invested = sum(allocation.values())
                print(f"    - Caixa: {(1-total_invested)*100:.2f}%")
            print(f"    - Retorno Máximo: {portfolio_return:.5f}")
        else:
            print("  - Justificativa: As restrições são contraditórias e o conjunto de soluções é VAZIO.")

    print("\n" + "=" * 80)

      EXERCÍCIO 3: ANÁLISE DE SISTEMAS LINEARES INTERVALARES


--- Sistema (A) ---

[CASO 1: [A]x ⊆ [b]] - Otimização Linear
  - Status: Feasible
  - Envoltório (Hull): [(0.0, 36.0), (12.0, 80.0)]
  - Justificativa: O problema de otimização tem uma região viável não vazia, e este 'hull' representa os limites (min/max) dessa região.

[CASO 2: [b] ⊆ [A]x] - Verificação de Viabilidade
  - Status: Feasible
  - Justificativa: O conjunto de soluções NÃO é vazio. Como a pergunta era apenas 'sim/não' (viabilidade), o status 'Feasible' é a resposta completa e não há um 'hull' para mostrar.

[CASO 3: [A]x = [b]] - Conclusão Lógica
  - Status: Verificar Interseção
  - Justificativa: A solução é a interseção do Caso 1 e Caso 2. Como o Caso 2 resultou em um conjunto vazio ('Infeasible'), a interseção também é VAZIA.

[CASO 4: [A]x ∩ [b] ≠ ∅] - Otimização Não-Linear
  - Status: Feasible
  - Envoltório (Hull): [(-120.0, 60.0), (-60.0, 240.0)]
  - Justificativa: O problema de otimização não-linear enc